In [4]:
!pip install tenseal

     |████████████████████████████████| 4.9 MB 12.7 MB/s 


In [48]:
import tenseal as ts
import random
from time import time

In [45]:
context = ts.context(ts.SCHEME_TYPE.BFV, poly_modulus_degree=4096, plain_modulus=1032193)

In [50]:
class Server:
  def __init__(self):
    pass
  
  def process_guess(k_pub, solution, guess):
    '''
    Note: I have been trying to perform this calculation using multiplication, but
    the only way I can think of is to do a multiplication in mod26 and check the
    remainder to see if the guesses are in the correct spot. However, I do not
    know how to perform the modulus multiplication (or if it is even supported)
    in the current SEAL library.
    '''
    print('\tComputations starting here are performed on the SERVER.')
    print('\tServer view of solution: {}'.format(solution))
    print('\tServer view of guess: {}'.format(guess))
    result1 = solution - guess[0]
    result2 = solution - guess[1]
    result3 = solution - guess[2]
    result4 = solution - guess[3]
    result5 = solution - guess[4]
    t_end = time()
    print('Computations starting here are NO LONGER performed on the SERVER.')
    return (result1, result2, result3, result4, result5)

class Client:
  def __init__(self, mod_degree=4096, plain_mod=1032193, words=['hello','world']):
    '''
    set up of encryption parameters using BFV scheme
    '''
    self.context = ts.context(ts.SCHEME_TYPE.BFV, poly_modulus_degree=mod_degree, plain_modulus=plain_mod)
    self.public_context = ts.context(ts.SCHEME_TYPE.BFV, poly_modulus_degree=mod_degree, plain_modulus=plain_mod)
    self.k_pr = self.public_context.secret_key()
    self.public_context.make_context_public()
    
    self.word_list = words

  def split_and_encrypt(self, guess):
    g1 = ts.bfv_vector(self.context, [guess[0]] * 5)
    g2 = ts.bfv_vector(self.context, [guess[1]] * 5)
    g3 = ts.bfv_vector(self.context, [guess[2]] * 5)
    g4 = ts.bfv_vector(self.context, [guess[3]] * 5)
    g5 = ts.bfv_vector(self.context, [guess[4]] * 5)
    return (g1, g2, g3, g4, g5)
  
  def process_and_decrypt(self, encrypted):
    return [encrypted[0].decrypt(), encrypted[1].decrypt(), encrypted[2].decrypt(), encrypted[3].decrypt(), encrypted[4].decrypt()]

  def play(self):
    while True:
      #play game loop
      print(' x  represents that "x" is in the CORRECT spot.')
      print('[x] represents that "x" is in the word, but NOT the correct spot.')
      print('(x) represents that "x" is NOT in the word.')
      print('Please only guess 5-letter words!')
      print('Start game? (y/n):')
      resp = input()
      resp = resp.lower()
      if resp == 'n':
        print('Thanks for playing!')
        break
      elif resp == 'y':
        print('Computations starting here are performed on the CLIENT.')
        #the solution word and guesses are assumed to be perform from the client
        #calculations are assumed to be performed on the side of the server
        solution = random.choice(self.word_list)
        solution_ascii = [] #get int-value of the word, the first plaintext vector
        for c in solution:
          solution_ascii.append(ord(c))
        print('Client view of plaintext ascii solution: {}'.format(solution_ascii))

        #encrypt the vector using the BFV scheme
        solution_encrypted = ts.bfv_vector(self.context, solution_ascii)
        
        #start guessing loop
        while True:
          print('Please enter your guess: ', end='')
          guess = input()
          guess = guess.lower()
          guess_ascii = []
          for c in guess: #get int-value of the guess, the second plaintext vector
            guess_ascii.append(ord(c))
          #Start timing of encrypted computations
          t_start = time()
          guess_encrypted = self.split_and_encrypt(guess_ascii)

          #mimic sending to server and process
          response_encrypted = Server.process_guess(self.public_context, solution_encrypted, guess_encrypted)
          print('Client view of encrypted response: {}'.format(response_encrypted))
          response = self.process_and_decrypt(response_encrypted)
          #End timing of encrypted computations
          t_end = time()
          print('\tTotal computations took: {} ms'.format((t_end-t_start)*1000))
          output = ''
          total = 0
          for i in range(5):
            inner_total = 0
            if response[i][i] == 0: #if the character is in the right spot
              output += '{} '.format(guess[i])
              total += 1
            else:
              for j in range(5):
                if response[i][j] == 0: #check remaining spots
                  if inner_total == 0:
                    output += '[{}]'.format(guess[i])
                  inner_total += 1
              if inner_total == 0:
                output += '({})'.format(guess[i])
          print('Response from server: {}'.format(output))
          if total >= 5:
            print('CONGRATULATIONS! You guessed the word!')
            break
      else:
        print('Invalid input, please enter "y" or "n"!')

if __name__ == '__main__':
  S = Server()
  C = Client()
  C.play()

 x  represents that "x" is in the CORRECT spot.
[x] represents that "x" is in the word, but NOT the correct spot.
(x) represents that "x" is NOT in the word.
Please only guess 5-letter words!
Start game? (y/n):
y
Computations starting here are performed on the CLIENT.
Client view of plaintext ascii solution: [119, 111, 114, 108, 100]
Please enter your guess: start
	Computations starting here are performed on the SERVER.
	Server view of solution: <tenseal.tensors.bfvvector.BFVVector object at 0x7f839fd4e750>
	Server view of guess: (<tenseal.tensors.bfvvector.BFVVector object at 0x7f839fd4e590>, <tenseal.tensors.bfvvector.BFVVector object at 0x7f83a056e650>, <tenseal.tensors.bfvvector.BFVVector object at 0x7f83a056e1d0>, <tenseal.tensors.bfvvector.BFVVector object at 0x7f83a056e350>, <tenseal.tensors.bfvvector.BFVVector object at 0x7f83a056e950>)
Computations starting here are NO LONGER performed on the SERVER.
Client view of encrypted response: (<tenseal.tensors.bfvvector.BFVVector obje